In [ ]:
import polars as pl
from pathlib import Path

In [ ]:
dpath = Path('../../dataset')

dtype = 'small'
articles = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/articles.parquet')

behaviors_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/behaviors.parquet')
history_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/history.parquet')

behaviors_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/behaviors.parquet')
history_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/history.parquet')

In [ ]:
from polimi.utils._custom import load_sparse_csr

ner_path = dpath.joinpath('urm').joinpath('ner').joinpath('small')
URM_train = load_sparse_csr(ner_path.joinpath('URM_train.npz'))
URM_val =  load_sparse_csr(ner_path.joinpath('URM_validation.npz'))

In [ ]:
from RecSys_Course_AT_PoliMi.Evaluation.Evaluator import EvaluatorHoldout

cutoff = 10
metric = 'NDCG'
evaluator = EvaluatorHoldout(URM_val, cutoff_list=[cutoff], exclude_seen=False)

In [ ]:
from RecSys_Course_AT_PoliMi.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

rec_instance = PureSVDRecommender(URM_train)
params = {'num_factors': 577}
rec_instance.fit(**params)
result_df, _ = evaluator.evaluateRecommender(rec_instance)
result_df.loc[cutoff][metric.upper()]

In [ ]:
from RecSys_Course_AT_PoliMi.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

rec_instance = UserKNNCFRecommender(URM_train)
params = {'topK': 467, 'shrink': 95, 'similarity': 'dice', 'normalize': True}
rec_instance.fit(**params)
result_df, _ = evaluator.evaluateRecommender(rec_instance)
result_df.loc[cutoff][metric.upper()]

In [ ]:
from polimi.utils._urm import _build_batch_ner_interactions, build_articles_with_processed_ner, build_user_id_mapping, build_ner_mapping
ap = build_articles_with_processed_ner(articles)
user_id_mapping = build_user_id_mapping(history_train.vstack(history_val))
ner_mapping = build_ner_mapping(ap)
ner_interactions = _build_batch_ner_interactions(history_train, ap, user_id_mapping, ner_mapping, 'article_id_fixed')